## Parte 1: Batch Scoring - Predicciones por Lotes

### 1.1 Cargar Modelo desde MLflow Registry

In [ ]:
import mlflow
import mlflow.pyfunc
from pyspark.sql import SparkSession
from pyspark.sql.functions import struct, col
import pandas as pd

# Configurar MLflow
mlflow.set_registry_uri("databricks")

# Nombre del modelo y versión (ajustar según Lab 4)
model_name = "diabetes_predictor"  # Cambiar según tu modelo
model_version = 1  # O "Production" para usar el stage

# Cargar modelo
model_uri = f"models:/{model_name}/{model_version}"
print(f"Cargando modelo desde: {model_uri}")

loaded_model = mlflow.pyfunc.load_model(model_uri)
print(f"✓ Modelo cargado exitosamente")

### 1.2 Preparar Datos de Entrada para Batch Scoring

In [ ]:
# Crear datos de ejemplo para predicciones
sample_data = pd.DataFrame({
    'age': [45, 52, 38, 61, 29, 47, 55, 42, 36, 58],
    'bmi': [27.3, 32.1, 24.5, 28.9, 22.4, 29.5, 31.8, 26.2, 23.7, 30.4],
    'blood_pressure': [120, 135, 110, 140, 115, 125, 138, 118, 112, 142],
    'glucose': [95, 160, 88, 175, 82, 105, 155, 92, 85, 168]
})

display(sample_data)

### 1.3 Realizar Predicciones Básicas

In [ ]:
# Predicciones con pandas DataFrame
predictions = loaded_model.predict(sample_data)

# Agregar predicciones al DataFrame
result_df = sample_data.copy()
result_df['prediction'] = predictions
result_df['prediction_timestamp'] = pd.Timestamp.now()

print(f"Total de predicciones: {len(result_df)}")
display(result_df)

### 1.4 Batch Scoring a Gran Escala con Spark UDF

In [ ]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType

# Crear Spark DataFrame
spark_df = spark.createDataFrame(sample_data)

# Definir UDF para predicciones distribuidas
@pandas_udf(DoubleType())
def predict_udf(*cols):
    # Reconstruir DataFrame desde columnas
    input_df = pd.concat(cols, axis=1)
    input_df.columns = sample_data.columns
    # Hacer predicción
    return pd.Series(loaded_model.predict(input_df))

# Aplicar predicciones
predictions_df = spark_df.withColumn(
    "prediction",
    predict_udf(*[col(c) for c in sample_data.columns])
)

display(predictions_df)

### 1.5 Guardar Resultados en Delta Lake

In [ ]:
# Definir ruta de salida
output_path = "/tmp/predictions/diabetes/batch_scoring"

# Guardar en Delta Lake
predictions_df.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .save(output_path)

print(f"✓ Predicciones guardadas en: {output_path}")

# Verificar datos guardados
saved_df = spark.read.format("delta").load(output_path)
print(f"Total de registros guardados: {saved_df.count()}")

### 1.6 Registrar Métricas de Batch Job

In [ ]:
from datetime import datetime

# Contar predicciones generadas
prediction_count = predictions_df.count()

# Registrar métricas en MLflow
with mlflow.start_run(run_name="batch_scoring_metrics"):
    mlflow.log_metric("predictions_count", prediction_count)
    mlflow.log_metric("execution_timestamp", datetime.now().timestamp())
    mlflow.log_param("model_name", model_name)
    mlflow.log_param("model_version", model_version)
    mlflow.log_param("output_path", output_path)
    
print(f"✓ Métricas registradas en MLflow")
print(f"  - Predicciones: {prediction_count}")
print(f"  - Timestamp: {datetime.now()}")

## Parte 2: Despliegue de Endpoints en Tiempo Real

### 2.1 Crear Endpoint REST con MLflow

In [ ]:
from mlflow.deployments import get_deploy_client

# Configurar cliente de despliegue
client = get_deploy_client("databricks")

# Configuración del endpoint
endpoint_name = "diabetes-predictor-endpoint"

endpoint_config = {
    "served_models": [{
        "model_name": model_name,
        "model_version": str(model_version),
        "workload_size": "Small",  # Small, Medium, Large
        "scale_to_zero_enabled": True  # Escalar a 0 cuando no hay tráfico
    }]
}

# Crear endpoint
try:
    endpoint = client.create_endpoint(
        name=endpoint_name,
        config=endpoint_config
    )
    print(f"✓ Endpoint creado: {endpoint_name}")
except Exception as e:
    print(f"Endpoint ya existe o error: {e}")
    print("Continuando con el endpoint existente...")

### 2.2 Verificar Estado del Endpoint

In [ ]:
import time

# Esperar a que el endpoint esté listo
print("Verificando estado del endpoint...")
max_wait = 300  # 5 minutos
wait_interval = 10
elapsed = 0

while elapsed < max_wait:
    try:
        endpoint_details = client.get_endpoint(endpoint_name)
        state = endpoint_details.get('state', {})
        
        if state.get('ready') == 'READY':
            print(f"\n✓ Endpoint listo para recibir tráfico")
            print(f"Estado: {state}")
            break
        else:
            print(f"Esperando... ({elapsed}s/{max_wait}s) - Estado: {state.get('ready', 'UNKNOWN')}")
            time.sleep(wait_interval)
            elapsed += wait_interval
    except Exception as e:
        print(f"Error al verificar estado: {e}")
        break

if elapsed >= max_wait:
    print(f"⚠️ Tiempo de espera agotado. El endpoint puede tardar más en estar listo.")

### 2.3 Consumir Endpoint REST

In [ ]:
import requests
import json

# Configuración
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

endpoint_url = f"https://{workspace_url}/serving-endpoints/{endpoint_name}/invocations"

# Datos de prueba
test_data = {
    "dataframe_records": [
        {"age": 45, "bmi": 27.3, "blood_pressure": 120, "glucose": 95},
        {"age": 52, "bmi": 32.1, "blood_pressure": 135, "glucose": 160}
    ]
}

# Headers
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Realizar petición
start_time = time.time()
response = requests.post(
    endpoint_url,
    headers=headers,
    json=test_data
)
latency = (time.time() - start_time) * 1000  # ms

# Procesar respuesta
if response.status_code == 200:
    predictions = response.json()
    print("✓ Predicciones recibidas:")
    print(json.dumps(predictions, indent=2))
    print(f"\nLatencia: {latency:.2f} ms")
else:
    print(f"✗ Error: {response.status_code}")
    print(response.text)

## Parte 3: Monitoreo y Logging de Predicciones

### 3.1 Implementar Logging de Predicciones

In [ ]:
import uuid
from datetime import datetime

class PredictionLogger:
    """Logger para predicciones con metadata completa"""
    
    def __init__(self, log_table_path, model_name, model_version):
        self.log_table_path = log_table_path
        self.model_name = model_name
        self.model_version = model_version
    
    def log_prediction(self, input_data, prediction, latency_ms, request_id=None):
        """Registra predicción con metadata"""
        if request_id is None:
            request_id = str(uuid.uuid4())
        
        # Preparar log entry
        log_entry = {
            'request_id': request_id,
            'timestamp': datetime.now(),
            'model_name': self.model_name,
            'model_version': str(self.model_version),
            'prediction': float(prediction),
            'latency_ms': latency_ms,
            **input_data
        }
        
        # Guardar en Delta Lake
        log_df = spark.createDataFrame([log_entry])
        log_df.write.format("delta").mode("append").save(self.log_table_path)
        
        return request_id

# Crear logger
log_path = "/tmp/ml-monitoring/prediction-logs"
logger = PredictionLogger(log_path, model_name, model_version)

print(f"✓ Logger configurado en: {log_path}")

### 3.2 Generar Tráfico de Prueba con Logging

In [ ]:
import numpy as np

# Generar tráfico sintético
print("Generando tráfico de prueba...\n")

num_requests = 20
for i in range(num_requests):
    # Generar datos de entrada
    test_input = {
        'age': int(np.random.uniform(25, 75)),
        'bmi': round(np.random.uniform(18, 40), 1),
        'blood_pressure': int(np.random.uniform(100, 160)),
        'glucose': int(np.random.uniform(70, 200))
    }
    
    # Realizar predicción
    start = time.time()
    pred_input = pd.DataFrame([test_input])
    prediction = loaded_model.predict(pred_input)[0]
    latency = (time.time() - start) * 1000
    
    # Registrar en log
    req_id = logger.log_prediction(test_input, prediction, latency)
    
    if i % 5 == 0:
        print(f"Request {i+1}/{num_requests}: Prediction={prediction:.3f}, Latency={latency:.1f}ms")

print(f"\n✓ Tráfico de prueba completado: {num_requests} predicciones")

### 3.3 Analizar Logs de Predicciones

In [ ]:
# Leer logs
logs_df = spark.read.format("delta").load(log_path)

print(f"📊 Total de predicciones registradas: {logs_df.count()}")

# Convertir a pandas para análisis
logs_pd = logs_df.toPandas()

# Estadísticas de latencia
print(f"\n📈 Estadísticas de Latencia:")
print(f"  Media: {logs_pd['latency_ms'].mean():.2f} ms")
print(f"  Mediana (P50): {logs_pd['latency_ms'].quantile(0.5):.2f} ms")
print(f"  P95: {logs_pd['latency_ms'].quantile(0.95):.2f} ms")
print(f"  P99: {logs_pd['latency_ms'].quantile(0.99):.2f} ms")
print(f"  Min: {logs_pd['latency_ms'].min():.2f} ms")
print(f"  Max: {logs_pd['latency_ms'].max():.2f} ms")

# Estadísticas de predicciones
print(f"\n🎯 Estadísticas de Predicciones:")
print(f"  Media: {logs_pd['prediction'].mean():.3f}")
print(f"  Std Dev: {logs_pd['prediction'].std():.3f}")
print(f"  Min: {logs_pd['prediction'].min():.3f}")
print(f"  Max: {logs_pd['prediction'].max():.3f}")

# Mostrar muestra de logs
display(logs_df.orderBy(col("timestamp").desc()).limit(10))

### 3.4 Visualizar Métricas de Monitoreo

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo
sns.set_style("whitegrid")

# Crear dashboard de monitoreo
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribución de latencia
axes[0, 0].hist(logs_pd['latency_ms'], bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(logs_pd['latency_ms'].mean(), color='red', linestyle='--', label='Media')
axes[0, 0].axvline(logs_pd['latency_ms'].quantile(0.95), color='orange', linestyle='--', label='P95')
axes[0, 0].set_title('Distribución de Latencia', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Latencia (ms)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].legend()

# 2. Distribución de predicciones
axes[0, 1].hist(logs_pd['prediction'], bins=25, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_title('Distribución de Predicciones', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Valor Predicho')
axes[0, 1].set_ylabel('Frecuencia')

# 3. Predicciones vs Edad
axes[1, 0].scatter(logs_pd['age'], logs_pd['prediction'], alpha=0.6, s=50)
axes[1, 0].set_title('Predicciones vs Edad', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Edad')
axes[1, 0].set_ylabel('Predicción')

# 4. Predicciones vs BMI
axes[1, 1].scatter(logs_pd['bmi'], logs_pd['prediction'], alpha=0.6, s=50, color='purple')
axes[1, 1].set_title('Predicciones vs BMI', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('BMI')
axes[1, 1].set_ylabel('Predicción')

plt.tight_layout()
plt.savefig('/tmp/monitoring_dashboard.png', dpi=100, bbox_inches='tight')
display(plt.gcf())

# Registrar dashboard en MLflow
with mlflow.start_run(run_name="monitoring_dashboard"):
    mlflow.log_artifact('/tmp/monitoring_dashboard.png')
    mlflow.log_metric("total_predictions", len(logs_pd))
    mlflow.log_metric("avg_latency_ms", logs_pd['latency_ms'].mean())
    mlflow.log_metric("p95_latency_ms", logs_pd['latency_ms'].quantile(0.95))

print("\n✓ Dashboard generado y registrado en MLflow")

## Parte 4: Detección de Data Drift

### 4.1 Implementar Detección de Drift con KS Test

In [ ]:
from scipy import stats

def calculate_drift(reference_data, current_data, features, threshold=0.05):
    """
    Detecta drift usando Kolmogorov-Smirnov test
    
    Args:
        reference_data: DataFrame con datos de referencia (training)
        current_data: DataFrame con datos actuales (producción)
        features: Lista de features a analizar
        threshold: P-value threshold para detectar drift
    
    Returns:
        Dict con resultados de drift por feature
    """
    drift_results = {}
    
    for feature in features:
        # KS test
        statistic, p_value = stats.ks_2samp(
            reference_data[feature],
            current_data[feature]
        )
        
        # Drift detectado si p-value < threshold
        drift_results[feature] = {
            'statistic': statistic,
            'p_value': p_value,
            'drift_detected': p_value < threshold,
            'severity': 'HIGH' if p_value < 0.01 else 'MEDIUM' if p_value < threshold else 'LOW'
        }
    
    return drift_results

print("✓ Función de detección de drift implementada")

### 4.2 Comparar Datos de Training vs Producción

In [ ]:
# Datos de referencia (simulados - en producción vendrían del training set)
reference_data = pd.DataFrame({
    'age': np.random.normal(50, 10, 1000),
    'bmi': np.random.normal(28, 5, 1000),
    'blood_pressure': np.random.normal(130, 15, 1000),
    'glucose': np.random.normal(120, 30, 1000)
})

# Datos de producción
production_data = logs_pd[['age', 'bmi', 'blood_pressure', 'glucose']]

# Detectar drift
features = ['age', 'bmi', 'blood_pressure', 'glucose']
drift_results = calculate_drift(reference_data, production_data, features, threshold=0.05)

# Mostrar resultados
print("🔍 Análisis de Data Drift:\n")
print(f"{'Feature':<20} {'P-Value':<12} {'Statistic':<12} {'Estado':<15} {'Severidad'}")
print("-" * 75)

for feature, result in drift_results.items():
    status = "⚠️ DRIFT" if result['drift_detected'] else "✓ Sin Drift"
    print(f"{feature:<20} {result['p_value']:<12.4f} {result['statistic']:<12.4f} {status:<15} {result['severity']}")

# Contar features con drift
drift_count = sum(1 for r in drift_results.values() if r['drift_detected'])
print(f"\nResumen: {drift_count}/{len(features)} features con drift detectado")

### 4.3 Visualizar Drift por Feature

In [ ]:
# Visualizar comparación de distribuciones
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, feature in enumerate(features):
    # Histogramas superpuestos
    axes[idx].hist(reference_data[feature], bins=30, alpha=0.5, label='Training (Reference)', 
                   edgecolor='black', density=True)
    axes[idx].hist(production_data[feature], bins=30, alpha=0.5, label='Production (Current)', 
                   edgecolor='black', density=True, color='orange')
    
    # Título con información de drift
    drift_info = drift_results[feature]
    status = "⚠️ DRIFT DETECTADO" if drift_info['drift_detected'] else "✓ Sin Drift"
    axes[idx].set_title(f'{feature}\n{status} (p={drift_info["p_value"]:.4f})', 
                       fontsize=11, fontweight='bold')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Densidad')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/tmp/drift_analysis.png', dpi=100, bbox_inches='tight')
display(plt.gcf())

print("✓ Visualización de drift generada")

## Parte 5: Sistema de Alertas

### 5.1 Configurar Sistema de Alertas

In [ ]:
class AlertSystem:
    """Sistema de alertas para monitoreo de modelos"""
    
    def __init__(self, thresholds):
        self.thresholds = thresholds
        self.alerts = []
    
    def check_latency(self, metrics):
        """Verificar latencia"""
        if metrics.get('p95_latency_ms', 0) > self.thresholds['max_latency_p95_ms']:
            self.alerts.append({
                'type': 'HIGH_LATENCY',
                'severity': 'WARNING',
                'message': f"Latencia P95 ({metrics['p95_latency_ms']:.1f}ms) excede threshold ({self.thresholds['max_latency_p95_ms']}ms)",
                'value': metrics['p95_latency_ms']
            })
    
    def check_drift(self, drift_results):
        """Verificar data drift"""
        drift_features = [f for f, r in drift_results.items() if r['drift_detected']]
        if drift_features:
            self.alerts.append({
                'type': 'DATA_DRIFT',
                'severity': 'WARNING',
                'message': f'Drift detectado en features: {", ".join(drift_features)}',
                'features': drift_features
            })
    
    def check_prediction_distribution(self, predictions, expected_mean, tolerance=0.2):
        """Verificar distribución de predicciones"""
        current_mean = predictions.mean()
        deviation = abs(current_mean - expected_mean) / expected_mean
        
        if deviation > tolerance:
            self.alerts.append({
                'type': 'PREDICTION_DISTRIBUTION_SHIFT',
                'severity': 'CRITICAL',
                'message': f'Media de predicciones ({current_mean:.3f}) difiere significativamente de la esperada ({expected_mean:.3f})',
                'deviation': deviation
            })
    
    def send_alerts(self):
        """Enviar alertas (simulado)"""
        if not self.alerts:
            print("✓ No hay alertas - Sistema operando normalmente")
            return
        
        print(f"\n🚨 {len(self.alerts)} ALERTA(S) DETECTADA(S):\n")
        for i, alert in enumerate(self.alerts, 1):
            print(f"{i}. [{alert['severity']}] {alert['type']}")
            print(f"   {alert['message']}\n")
        
        # En producción, aquí se integraría con:
        # - Email (SendGrid, SMTP)
        # - Slack webhook
        # - Microsoft Teams webhook
        # - PagerDuty
        # - Azure Monitor
    
    def get_alert_summary(self):
        """Resumen de alertas"""
        return {
            'total': len(self.alerts),
            'critical': sum(1 for a in self.alerts if a['severity'] == 'CRITICAL'),
            'warning': sum(1 for a in self.alerts if a['severity'] == 'WARNING')
        }

print("✓ Sistema de alertas implementado")

### 5.2 Ejecutar Verificaciones y Generar Alertas

In [ ]:
# Configurar thresholds
thresholds = {
    'max_latency_p95_ms': 200,
    'min_accuracy': 0.85,
    'max_drift_pvalue': 0.05
}

# Crear sistema de alertas
alert_system = AlertSystem(thresholds)

# Ejecutar verificaciones
metrics = {
    'p95_latency_ms': logs_pd['latency_ms'].quantile(0.95),
    'avg_latency_ms': logs_pd['latency_ms'].mean()
}

alert_system.check_latency(metrics)
alert_system.check_drift(drift_results)
alert_system.check_prediction_distribution(
    logs_pd['prediction'], 
    expected_mean=0.5,  # Ajustar según modelo
    tolerance=0.3
)

# Enviar alertas
alert_system.send_alerts()

# Mostrar resumen
summary = alert_system.get_alert_summary()
print(f"\n📊 Resumen de Alertas:")
print(f"  Total: {summary['total']}")
print(f"  Críticas: {summary['critical']}")
print(f"  Advertencias: {summary['warning']}")

## Parte 6: Buenas Prácticas - Model Tagging

### 6.1 Aplicar Tags al Modelo en Producción

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Tags recomendados para producción
production_tags = {
    # Metadata técnica
    "model_type": "regression",
    "framework": "sklearn",
    
    # Información del dataset
    "training_date": datetime.now().strftime("%Y-%m-%d"),
    "training_samples": "1000",
    
    # Deployment info
    "deployed_by": "data-science-team",
    "deployment_date": datetime.now().strftime("%Y-%m-%d"),
    "environment": "production",
    
    # Monitoring
    "monitoring_enabled": "true",
    "alert_threshold_latency_p95": "200",
    
    # Business context
    "use_case": "diabetes-prediction",
    "business_owner": "healthcare-analytics"
}

# Aplicar tags
try:
    for key, value in production_tags.items():
        client.set_model_version_tag(model_name, model_version, key, value)
    print(f"✓ {len(production_tags)} tags aplicados al modelo {model_name} v{model_version}")
except Exception as e:
    print(f"Error al aplicar tags: {e}")

# Verificar tags
try:
    model_version_info = client.get_model_version(model_name, model_version)
    print(f"\nTags actuales:")
    for key, value in model_version_info.tags.items():
        print(f"  {key}: {value}")
except Exception as e:
    print(f"No se pudieron verificar tags: {e}")

## Parte 7: Resumen y Reporte Final

### 7.1 Generar Reporte de Monitoreo

In [ ]:
# Generar reporte completo
report = f"""
{'='*80}
REPORTE DE MONITOREO - MODELO EN PRODUCCIÓN
{'='*80}

📋 INFORMACIÓN DEL MODELO
  Nombre: {model_name}
  Versión: {model_version}
  Endpoint: {endpoint_name}
  Fecha de reporte: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

📊 MÉTRICAS DE SERVICIO
  Total de predicciones: {len(logs_pd)}
  Latencia media: {logs_pd['latency_ms'].mean():.2f} ms
  Latencia P95: {logs_pd['latency_ms'].quantile(0.95):.2f} ms
  Latencia P99: {logs_pd['latency_ms'].quantile(0.99):.2f} ms

🎯 ESTADÍSTICAS DE PREDICCIONES
  Media: {logs_pd['prediction'].mean():.4f}
  Desviación estándar: {logs_pd['prediction'].std():.4f}
  Rango: [{logs_pd['prediction'].min():.4f}, {logs_pd['prediction'].max():.4f}]

🔍 DETECCIÓN DE DRIFT
  Features analizados: {len(features)}
  Features con drift: {sum(1 for r in drift_results.values() if r['drift_detected'])}
"""

# Agregar detalles de drift
for feature, result in drift_results.items():
    if result['drift_detected']:
        report += f"    ⚠️ {feature}: p-value={result['p_value']:.4f}\n"

report += f"""
🚨 ALERTAS
  Total de alertas: {summary['total']}
  Críticas: {summary['critical']}
  Advertencias: {summary['warning']}

✅ RECOMENDACIONES
"""

# Generar recomendaciones dinámicas
if summary['total'] == 0:
    report += "  - Sistema operando dentro de parámetros normales\n"
    report += "  - Continuar con monitoreo regular\n"
else:
    if summary['critical'] > 0:
        report += "  - ⚠️ ACCIÓN INMEDIATA REQUERIDA: Alertas críticas detectadas\n"
    if any(r['drift_detected'] for r in drift_results.values()):
        report += "  - Considerar reentrenamiento del modelo debido a drift\n"
    if logs_pd['latency_ms'].quantile(0.95) > thresholds['max_latency_p95_ms']:
        report += "  - Optimizar latencia o escalar recursos del endpoint\n"

report += f"""
{'='*80}
"""

print(report)

# Guardar reporte
report_path = "/tmp/monitoring_report.txt"
with open(report_path, 'w') as f:
    f.write(report)

# Registrar en MLflow
with mlflow.start_run(run_name="monitoring_report"):
    mlflow.log_artifact(report_path)
    mlflow.log_artifact('/tmp/monitoring_dashboard.png')
    mlflow.log_artifact('/tmp/drift_analysis.png')
    
    # Métricas principales
    mlflow.log_metrics({
        "total_predictions": len(logs_pd),
        "avg_latency_ms": logs_pd['latency_ms'].mean(),
        "p95_latency_ms": logs_pd['latency_ms'].quantile(0.95),
        "prediction_mean": logs_pd['prediction'].mean(),
        "drift_features_count": sum(1 for r in drift_results.values() if r['drift_detected']),
        "total_alerts": summary['total']
    })

print("\n✓ Reporte completo generado y registrado en MLflow")

## 🎉 Laboratorio Completado

### Has aprendido a:

✅ **Desplegar modelos** con batch scoring y endpoints REST  
✅ **Configurar monitoreo** de latencia y métricas de servicio  
✅ **Implementar logging** estructurado de predicciones  
✅ **Detectar data drift** usando pruebas estadísticas  
✅ **Configurar alertas** automatizadas para incidentes  
✅ **Aplicar buenas prácticas** de MLOps (tagging, documentación)  

### Próximos Pasos

1. **Automatizar monitoreo**: Crear job programado que ejecute este notebook diariamente
2. **Integrar alertas**: Conectar con Slack, Teams o email para notificaciones
3. **Implementar retraining**: Automatizar reentrenamiento cuando se detecte drift
4. **Escalar a producción**: Mover a entorno productivo con gobernanza completa

### Recursos Adicionales

- [Databricks Model Serving](https://docs.databricks.com/machine-learning/model-serving/index.html)
- [MLflow Deployments](https://mlflow.org/docs/latest/deployment/index.html)
- [Model Monitoring Best Practices](https://www.databricks.com/blog/2022/04/19/model-monitoring-best-practices.html)

## Limpieza (Opcional)

Ejecuta las siguientes celdas para limpiar recursos creados durante el laboratorio.

In [ ]:
# OPCIONAL: Eliminar endpoint para evitar costos
# Descomentar para ejecutar

# try:
#     client.delete_endpoint(endpoint_name)
#     print(f"✓ Endpoint {endpoint_name} eliminado")
# except Exception as e:
#     print(f"Error al eliminar endpoint: {e}")

In [ ]:
# OPCIONAL: Limpiar tablas temporales
# Descomentar para ejecutar

# import shutil
# try:
#     shutil.rmtree('/dbfs/tmp/predictions')
#     shutil.rmtree('/dbfs/tmp/ml-monitoring')
#     print("✓ Tablas temporales eliminadas")
# except Exception as e:
#     print(f"Error al limpiar: {e}")